create separate models for title prediction and/or text prediction and divide the dataset into title datasets and text datasets. for that need to redo clean_csv

Importing primary dependencies for data fetch and cleanup

In [3]:
import pandas as pd
import numpy as np
import re

from pandas import read_csv

Various constants and data fetch

In [6]:
max_features=10240
max_len=2048
test_prop=0.3
epoch_count=100
batch_size=512

In [72]:
fake=read_csv('https://raw.githubusercontent.com/MobiusYeeitus/FakeNewsDetection/master/datasets/set1/fake.csv')
real=read_csv('https://raw.githubusercontent.com/MobiusYeeitus/FakeNewsDetection/master/datasets/set1/true.csv')
data2=read_csv('https://raw.githubusercontent.com/MobiusYeeitus/FakeNewsDetection/master/datasets/cleaned/ds2.csv')

In [73]:
fake.tail(4)

,title,text,subject,date
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016"


In [74]:
real.tail(4)

,title,text,subject,date
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017"


Compressed every cleanup job into the *clean_csv* function

In [176]:

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
stem=PorterStemmer()

from gensim.parsing.preprocessing import remove_stopwords
pd.options.mode.chained_assignment = None

In [90]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Need to markdown sequence and omit stopwords using nltk

In [205]:
def clean_string(string): #function to clean a string | to be used in clean_csv
    blank=''
    string=string.lower() #convert string to lowercase
    string=re.sub('\n|\r|\t','',string) #cleaning of whitespaces
    string=re.sub(r'[^\w\s]+','',string) #cleaning of symbols
    remove_stopwords(string)
    return string
def clean_csv(df,identity): #cleaning the CSV file
    print('Cleaning CSV.')
    df=df.dropna() #drop all tuples with empty attributes
    length=[]
    [length.append(len(str(text))) for text in df["text"]] #calculate length of an article
    df['length'] = length #saves the length of the artivle in a new column
    df = df.drop(df['text'][df['length'] < 120].index, axis = 0) #drop all tuples with less than 120 letters
    df['text']=df['title']+" "+df['text'] #append text to title
    vfunc=np.vectorize(clean_string) #vectorize th clean_string function for use in pandas df
    df['text']=vfunc(df['text']) #apply the vectorized function to further clean the data
    df['identity']=identity #add the identity (T/F) or (0/1) in a new attribute to the data
    df.reset_index() #reset all indexes of the dataset
    for x in range (len(df)):
        df['text'].iloc[x]=remove_stopwords(df['text'].iloc[x])
    print('Cleaning completed.')
    return df
def stemming(string): #not using stemming, reduces accuracy with nouns
    review = re.sub('[^a-zA-Z]',' ',string)
    review = review.lower()
    review = review.split()
    review = [stem.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    return review


In [209]:
def clean_csv2(df): #cleaning the CSV file
    print('Cleaning CSV.')
    df=df.dropna() #drop all tuples with empty attributes
    length=[]
    [length.append(len(str(text))) for text in df["text"]] #calculate length of an article
    df['length'] = length #saves the length of the artivle in a new column
    df = df.drop(df['text'][df['length'] < 120].index, axis = 0) #drop all tuples with less than 120 letters
    #df['text']=df['title']+" "+df['text'] #append text to title
    vfunc=np.vectorize(clean_string) #vectorize th clean_string function for use in pandas df
    df['text']=vfunc(df['text']) #apply the vectorized function to further clean the data
    df.reset_index() #reset all indexes of the dataset
    for x in range (len(df)):
        df['text'].iloc[x]=remove_stopwords(df['text'].iloc[x])
    print('Cleaning completed.')
    return df

In [211]:
data21=data2.iloc[:int(len(fake)/2),:]
data22=data2.iloc[int(len(fake)/2):,:]

In [213]:
data21=clean_csv2(data21)

Cleaning CSV.


MemoryError: Unable to allocate 4.65 GiB for an array with shape (11168,) and data type <U111784

Splitting up the datasets to conserve RAM, and aviod crashes during cleaning

In [77]:
fake1=fake.iloc[:int(len(fake)/2),:]
fake2=fake.iloc[int(len(fake)/2):,:]

In [177]:
fake1=clean_csv(fake1,0)

Cleaning CSV.
Cleaning completed.


In [179]:
fake1.tail(4)

,title,text,subject,date,length,identity
11734,TRUMP IS REPORTEDLY Preparing Executive Order ...,trump reportedly preparing executive order dep...,politics,"Feb 4, 2017",2560,0
11735,HOW LOS ANGELES COUNTY Helps Illegal Aliens Ge...,los angeles county helps illegal aliens free b...,politics,"Feb 4, 2017",1736,0
11737,MAXINE WATERS TELLS THE “GREATEST DESIRE” FOR ...,maxine waters tells greatest desire nasty dems...,politics,"Feb 4, 2017",1223,0
11738,MMA FIGHTER JAKE SHIELDS Embarrasses Cowards I...,mma fighter jake shields embarrasses cowards m...,politics,"Feb 4, 2017",3859,0


In [180]:
fake2=clean_csv(fake2,0)

Cleaning CSV.
Cleaning completed.


In [181]:
fake=pd.concat([fake1,fake2])

In [182]:
fake.tail(4)

,title,text,subject,date,length,identity
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,justice yahoo settles email privacy classactio...,Middle-east,"January 16, 2016",1684,0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,sunnistan allied safe zone plan territorial bo...,Middle-east,"January 15, 2016",25065,0
23479,How to Blow $700 Million: Al Jazeera America F...,blow 700 million al jazeera america finally ca...,Middle-east,"January 14, 2016",2685,0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,10 navy sailors held iranian military signs ne...,Middle-east,"January 12, 2016",5251,0


In [183]:
real1=real.iloc[:int(len(real)/2),:]
real2=real.iloc[int(len(real)/2):,:]

In [184]:
real1=clean_csv(real1,1)

Cleaning CSV.
Cleaning completed.


In [185]:
real2=clean_csv(real2,1)

Cleaning CSV.
Cleaning completed.


In [186]:
real=pd.concat([real1,real2])

In [187]:
real.tail(4)

,title,text,subject,date,length,identity
21413,LexisNexis withdrew two products from Chinese ...,lexisnexis withdrew products chinese market lo...,worldnews,"August 22, 2017",800,1
21414,Minsk cultural hub becomes haven from authorities,minsk cultural hub haven authorities minsk reu...,worldnews,"August 22, 2017",1950,1
21415,Vatican upbeat on possibility of Pope Francis ...,vatican upbeat possibility pope francis visiti...,worldnews,"August 22, 2017",1199,1
21416,Indonesia to buy $1.14 billion worth of Russia...,indonesia buy 114 billion worth russian jets j...,worldnews,"August 22, 2017",1338,1


Dropping all unnecessary columns

In [188]:
fake=fake[['text','identity']]
real=real[['text','identity']]

In [189]:
fake.tail(2)

,text,identity
23479,blow 700 million al jazeera america finally ca...,0
23480,10 navy sailors held iranian military signs ne...,0


In [199]:
data1=pd.concat([real,fake],ignore_index=True)

In [198]:
data1.tail(4)

,text,identity
43748,justice yahoo settles email privacy classactio...,0
43749,sunnistan allied safe zone plan territorial bo...,0
43750,blow 700 million al jazeera america finally ca...,0
43751,10 navy sailors held iranian military signs ne...,0


In [200]:
data=pd.concat([data1,data2],ignore_index=True)

In [201]:
data.tail(4)

,text,identity,Unnamed: 0
61810,nfl playoffs schedule matchups and odds the n...,0,18058.0
61811,macys is said to receive takeover approach by ...,0,18059.0
61812,nato russia to hold parallel exercises in balk...,1,18060.0
61813,what keeps the f35 alive david swanson is an...,1,18061.0


In [27]:
data=data[['text','identity']]
data.reset_index()
data.tail()

,text,identity
61809,rapper ti trump a poster child for white supre...,0
61810,nfl playoffs schedule matchups and odds the n...,0
61811,macys is said to receive takeover approach by ...,0
61812,nato russia to hold parallel exercises in balk...,1
61813,what keeps the f35 alive david swanson is an...,1


Importing dependencies for ANN and Representation

In [2]:
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Conv1D,MaxPool1D,Dropout,GRU
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score

Tokenizing the dataset for use in the ANN (One_Hot Representation)

In [29]:
print('Tokenizing DF.')
tokenizer=Tokenizer(num_words=max_features,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',split =' ') #filters is redundant, but still keeping if I missed any symbols previously
tokenizer.fit_on_texts(texts=data['text']) #fit
X=tokenizer.texts_to_sequences(texts=data['text'])
X=pad_sequences(sequences=X,maxlen=max_len,padding='pre') #padding the set of words
Y=data['identity'].values #identities (0/1)
V=tokenizer.word_index #vocabulary
print('Tokenized DF.')

Tokenizing DF.
Tokenized DF.


Creating the model and compiling it to be used.

In [ ]:
def create_lstm_model(max_features): #creating the internal structure of the simple LSTM model
        print('Creating LSTM model.')
        lstm_model=Sequential(name='Sierra')
        lstm_model.add(layer=Embedding(input_dim=max_features,output_dim=128,name='Echo'))
        lstm_model.add(layer=LSTM(units=128,name='Lima'))
        lstm_model.add(layer=Dense(1,activation='sigmoid',name ='Omega'))
        print('Model creation completed.')
        lstm_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
        print('Model compiled.')
        return lstm_model

In [ ]:
lstm_model=create_lstm_model(max_features)

Creating LSTM model.
Model creation completed.
Model compiled.


In [30]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y)

In [ ]:
lstm_model.fit(X_train,Y_train,batch_size=batch_size,epochs=epoch_count,verbose=1,validation_split=test_prop,shuffle=1,use_multiprocessing=True)

Epoch 1/100
64/64 [==============================] - 35s 497ms/step - loss: 0.5000 - accuracy: 0.7618 - val_loss: 0.2934 - val_accuracy: 0.8724
Epoch 2/100
64/64 [==============================] - 29s 454ms/step - loss: 0.2231 - accuracy: 0.9183 - val_loss: 0.2396 - val_accuracy: 0.9109
Epoch 3/100
64/64 [==============================] - 29s 448ms/step - loss: 0.1557 - accuracy: 0.9432 - val_loss: 0.2051 - val_accuracy: 0.9206
Epoch 4/100
64/64 [==============================] - 29s 461ms/step - loss: 0.3223 - accuracy: 0.8482 - val_loss: 0.3711 - val_accuracy: 0.8282
Epoch 5/100
64/64 [==============================] - 30s 469ms/step - loss: 0.2008 - accuracy: 0.9304 - val_loss: 0.2360 - val_accuracy: 0.9057
Epoch 6/100
64/64 [==============================] - 28s 446ms/step - loss: 0.1278 - accuracy: 0.9566 - val_loss: 0.2349 - val_accuracy: 0.9262
Epoch 7/100
64/64 [==============================] - 29s 449ms/step - loss: 0.1152 - accuracy: 0.9644 - val_loss: 0.1919 - val_accuracy:

In [ ]:
Y_pred=(lstm_model.predict(X_test)>=0.5).astype(int)

483/483 [==============================] - 12s 25ms/step


In [ ]:
accuracy_score(Y_test,Y_pred)

0.9437038954316035

In [ ]:
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95      8159
           1       0.94      0.95      0.94      7295

    accuracy                           0.94     15454
   macro avg       0.94      0.94      0.94     15454
weighted avg       0.94      0.94      0.94     15454



In [ ]:
lstm_model.save('models/ds1-2_combined_epoch-100')

In [31]:
nmod=tf.keras.saving.load_model('models/ds1-2_combined_epoch-100_extensive')

In [32]:
y_pred = (nmod.predict(X_test) >=0.5).astype(int)

483/483 [==============================] - 186s 384ms/step


In [33]:
accuracy_score(Y_test,y_pred)

0.9732108192053838

In [39]:
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      8132
           1       0.97      0.97      0.97      7322

    accuracy                           0.97     15454
   macro avg       0.97      0.97      0.97     15454
weighted avg       0.97      0.97      0.97     15454



In [40]:
string_predict1=pd.DataFrame()
string_predict1=data.iloc[22000:23000]
string_predict1

,text,identity
22000,trumps dc hotel raised price and made million...,0
22001,trumps rhetoric just resulted in a trillion d...,0
22002,this old tweet from trump reveals the real re...,0
22003,trump is raising your healthcare premiums and...,0
22004,donald trump just announced another position ...,0
...,...,...
22995,watch teacher whitesplains his use of ner to ...,0
22996,trump accidentally says he wants single payer...,0
22997,trump nominee brags that taking away healthca...,0
22998,trump says hes going to save the country mone...,0


In [43]:
X=tokenizer.texts_to_sequences(texts=string_predict1['text'])
X=pad_sequences(sequences=X,maxlen=max_len,padding='pre')
string_predict1_res=(nmod.predict(X)>=0.5).astype(int)

32/32 [==============================] - 12s 379ms/step


In [44]:
accuracy_score(string_predict1_res,string_predict1['identity'])

0.999

In [46]:
string_predict2=pd.DataFrame()
string_predict2=data.iloc[0:1000]
string_predict2


,text,identity
0,as us budget fight looms republicans flip thei...,1
1,us military to accept transgender recruits on ...,1
2,senior us republican senator let mr mueller do...,1
3,fbi russia probe helped by australian diplomat...,1
4,trump wants postal service to charge much more...,1
...,...,...
995,house panels seek documents on puerto rico uti...,1
996,trump releases some jfk files blocks others un...,1
997,tillerson tells myanmar army chief us concerne...,1
998,exclusive while advising trump in 2016 excia c...,1


In [48]:
X=tokenizer.texts_to_sequences(texts=string_predict2['text'])
X=pad_sequences(sequences=X,maxlen=max_len,padding='pre')
string_predict2_res=(nmod.predict(X)>=0.5).astype(int)

32/32 [==============================] - 12s 380ms/step


In [49]:
accuracy_score(string_predict2_res,string_predict2['identity'])

0.989